In [1]:
import numpy as np
import pandas as pd
import os
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import gensim
from nltk.util import ngrams 
import emoji
from itertools import chain
paths = "./dataset/"

In [2]:
def get_one_two_three(selects):
    global All_selects
    thgrams_selects = {}
    bigrams_selects = {}
    selects_token ={}
    for select in selects:
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(select)
        tokens = [i for i in tokens if(i.lower() not in stopwords.words('english'))]
        bigrams_t = ngrams(tokens, 2) 
        thgrams_t = ngrams(tokens, 3)
        tokens = list(set(tokens))
        bigrams_t = list(set(bigrams_t))
        thgrams_t = list(set(thgrams_t))
        for t in thgrams_t:
            t = str(t).split("'")
            t = t[1] +" "+ t[3]+" "+t[5]
            if t not in thgrams_selects:
                thgrams_selects[t] = 1
            else:
                thgrams_selects[t] += 1
        for b in bigrams_t:
            b = str(b).split("'")
            b = b[1]+" "+b[3]
            if b not in bigrams_selects:
                bigrams_selects[b] = 1
            else:
                bigrams_selects[b] += 1
        for token in tokens:
            if token not in selects_token:
                selects_token[token] = 1
            else:
                selects_token[token] += 1
    df_selects_token = pd.DataFrame.from_dict(selects_token,orient="index",columns=["频率"]).sort_values(by="频率",ascending=False)
    df_selects_token = df_selects_token[df_selects_token["频率"]>=3]
    df_bigrams_selects = pd.DataFrame.from_dict(bigrams_selects,orient="index",columns=["频率"]).sort_values(by="频率",ascending=False)
    df_bigrams_selects = df_bigrams_selects[df_bigrams_selects["频率"]>=2]
    df_thgrams_selects = pd.DataFrame.from_dict(thgrams_selects,orient="index",columns=["频率"]).sort_values(by="频率",ascending=False)
    df_thgrams_selects = df_thgrams_selects[df_thgrams_selects["频率"]>=2]
    return df_selects_token,df_bigrams_selects,df_thgrams_selects

In [3]:
def get_df_out(titles,comments):
    df_titles_token,df_bigrams_titles,df_thgrams_titles = get_one_two_three(titles)
    df_comments_token,df_bigrams_comments,df_thgrams_comments = get_one_two_three(comments)
    df_titles_token['标题-1字']=df_titles_token.index
    df_titles_token = df_titles_token.reset_index(drop=True)
    df_bigrams_titles['标题-2字']=df_bigrams_titles.index
    df_bigrams_titles = df_bigrams_titles.reset_index(drop=True)
    df_thgrams_titles['标题-3字']=df_thgrams_titles.index
    df_thgrams_titles = df_thgrams_titles.reset_index(drop=True)
    df_comments_token['评论-1字']=df_comments_token.index
    df_comments_token = df_comments_token.reset_index(drop=True)
    df_bigrams_comments['评论-2字']=df_bigrams_comments.index
    df_bigrams_comments = df_bigrams_comments.reset_index(drop=True)
    df_thgrams_comments['评论-3字']=df_thgrams_comments.index
    df_thgrams_comments = df_thgrams_comments.reset_index(drop=True)
    if df_titles_token.empty:
        df_t = pd.DataFrame()
    elif df_bigrams_titles.empty:
        df_t = df_titles_token
    elif df_thgrams_titles.empty:
        df_t = pd.concat([df_titles_token,df_bigrams_titles],axis=1)
    else:
        df_t = pd.concat([df_titles_token,df_bigrams_titles,df_thgrams_titles],axis=1)
    if df_thgrams_comments.empty:
        df_c = pd.concat([df_comments_token,df_bigrams_comments],axis=1)
    else:
        df_c = pd.concat([df_comments_token,df_bigrams_comments,df_thgrams_comments],axis=1)
    df_out = pd.concat([df_t,df_c],axis=1)
    return df_out

In [17]:
def change_coloum(df):
    df.columns = [x+"-"+str(i) if x=="频率" else x for i,x in enumerate(df.columns)]
    a,b,c = [],[],[]
    for i,j in enumerate(list(df.columns)):
        if i%2==0:
            a.append(j)
        else:
            b.append(j)
    for i in range(int(len(list(df.columns))/2)):
        c.append(b[i])
        c.append(a[i])
    df = df[c]
    return df

In [18]:
df_tmp = pd.DataFrame()
for _,_,files in os.walk(paths):
    print(files)
writer = pd.ExcelWriter("final.xlsx")
all_titles = []
all_comments = []
for file in files:
    keyname = file.split("-")[0]
    path = paths+file
    df = pd.read_excel(path)
    df = df[["标题","内容"]]
    titles = []
    comments = []
    df.标题.apply(lambda x:titles.append(x))
    df.内容.apply(lambda x:comments.append(x))
    titles = [emoji.demojize(str(i)).lower() for i in titles]
    comments = [emoji.demojize(str(i)).lower() for i in comments]
    all_titles.append(titles)
    all_comments.append(comments)
    df_out = get_df_out(titles,comments)
    df_out = change_coloum(df_out)
    df_out.to_excel(writer,sheet_name=keyname+"-"+str(len(df)),index=None)
df_out = get_df_out(list(chain.from_iterable(all_titles)),list(chain.from_iterable(all_comments)))
df_out = change_coloum(df_out)
df_out.to_excel("all_titles_comments.xlsx",sheet_name="所有标题和评论词频汇总",index=None)
#     print(df_out.columns)
writer.close()
test = pd.read_excel("final.xlsx",None)
writer = pd.ExcelWriter("final-out.xlsx")
for key in test.keys():
    pd_out = test[key]
    pd_out = pd_out.head(100)
    pd_out.to_excel(writer,sheet_name=key,index=None)
writer.close()

['B01FDBZA82-US-Reviews-200924.xlsx', 'B01JG4E4ES-US-Reviews-200924.xlsx', 'B071L7VRP2-US-Reviews-200924.xlsx', 'B0779M3T5L-US-Reviews-200924.xlsx', 'B077ZDDKM7-US-Reviews-200924.xlsx', 'B07KVTD7Q6-US-Reviews-200924.xlsx', 'B07PVV2ZSL-US-Reviews-200924.xlsx', 'B07RWSNYR1-US-Reviews-200924.xlsx', 'B08CSGH83G-US-Reviews-200924.xlsx']


In [16]:
file = "B01FDBZA82-US-Reviews-200924.xlsx"
path = paths+file
df = pd.read_excel(path)
df = df[["标题","内容"]]
titles = []
comments = []
df.标题.apply(lambda x:titles.append(x))
df.内容.apply(lambda x:comments.append(x))
titles = [emoji.demojize(str(i)).lower() for i in titles]
comments = [emoji.demojize(str(i)).lower() for i in comments]
all_titles.append(titles)
all_comments.append(comments)
df_out = get_df_out(titles,comments)

df_out = change_coloum(df_out)
df_out

,标题-1字,频率-0,标题-2字,频率-2,标题-3字,频率-4,评论-1字,频率-6,评论-2字,频率-8,评论-3字,频率-10
0,stars,64.0,five stars,47.0,great plant stands,3.0,great,102,plant stands,48.0,love plant stands,5.0
1,great,60.0,plant stands,33.0,sturdy plant stands,3.0,plants,98,potted plants,19.0,sturdy enough hold,5.0
2,stands,49.0,four stars,11.0,sturdy well made,2.0,stands,93,great price,15.0,great plant stands,5.0
3,five,47.0,great product,10.0,product great price,2.0,sturdy,76,well made,13.0,heavy potted plants,4.0
4,plant,47.0,great price,8.0,nice plant stands,2.0,plant,75,great product,12.0,really like plant,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
306,NaN,NaN,NaN,NaN,NaN,NaN,wet,3,NaN,NaN,NaN,NaN
307,NaN,NaN,NaN,NaN,NaN,NaN,level,3,NaN,NaN,NaN,NaN
308,NaN,NaN,NaN,NaN,NaN,NaN,otherwise,3,NaN,NaN,NaN,NaN
309,NaN,NaN,NaN,NaN,NaN,NaN,fast,3,NaN,NaN,NaN,NaN


In [7]:
df_good = pd.DataFrame()
df_bad = pd.DataFrame()
df_none = pd.DataFrame()
for file in files:
    keyname = file.split("-")[0]
    path = paths+file
    df = pd.read_excel(path)
    df = df[["标题","内容","星级"]]
    df.loc[df["星级"].isnull(),"星级"] = 0
    df_good = pd.concat([df_good,df[df["星级"]>=4]])
    df_bad = pd.concat([df_bad,df[df["星级"].isin([1,2,3])]])
    df_none = pd.concat([df_none,df[df["星级"]==0]])
good_titles = [emoji.demojize(str(i)).lower() for i in df_good["标题"].to_list()]
good_comments = [emoji.demojize(str(i)).lower() for i in df_good["内容"].to_list()]
bad_titles = [emoji.demojize(str(i)).lower() for i in df_bad["标题"].to_list()]
bad_comments = [emoji.demojize(str(i)).lower() for i in df_bad["内容"].to_list()]
none_titles = [emoji.demojize(str(i)).lower() for i in df_none["标题"].to_list()]
none_comments = [emoji.demojize(str(i)).lower() for i in df_none["内容"].to_list()]

In [8]:
writer = pd.ExcelWriter("star-rating.xlsx")
good_df_out = get_df_out(good_titles,good_comments)
good_df_out.columns = [x+"-"+str(i) if x=="频率" else x for i,x in enumerate(good_df_out.columns)]
good_df_out = good_df_out[['标题-1字','频率-0','标题-2字','频率-2', '标题-3字', '频率-4','评论-1字','频率-6','评论-2字','频率-8','评论-3字','频率-10']]
good_df_out.to_excel(writer,sheet_name="好评-"+str(len(good_df_out)),index=None)
bad_df_out = get_df_out(bad_titles,bad_comments)
bad_df_out.columns = [x+"-"+str(i) if x=="频率" else x for i,x in enumerate(bad_df_out.columns)]
bad_df_out = bad_df_out[['标题-1字','频率-0','标题-2字','频率-2', '评论-1字','频率-4','评论-2字','频率-6','评论-3字','频率-8']]
bad_df_out.to_excel(writer,sheet_name="差评-"+str(len(bad_df_out)),index=None)
none_df_out = get_df_out(none_titles,none_comments)
none_df_out.columns = [x+"-"+str(i) if x=="频率" else x for i,x in enumerate(none_df_out.columns)]
none_df_out = none_df_out[['标题-1字','频率-0','标题-2字','频率-2','评论-1字','频率-4','评论-2字','频率-6']]
none_df_out.to_excel(writer,sheet_name="未评价-"+str(len(none_df_out)),index=None)
good_bad_none_all = pd.concat([good_df_out,bad_df_out,none_df_out])
df_tmp1 = good_bad_none_all[["频率-0","标题-1字"]].groupby("标题-1字").sum().sort_values(by="频率-0",ascending=False).reset_index()
df_tmp2 = good_bad_none_all[["频率-2","标题-2字"]].groupby("标题-2字").sum().sort_values(by="频率-2",ascending=False).reset_index()
df_tmp3 = good_bad_none_all[["频率-4","标题-3字"]].groupby("标题-3字").sum().sort_values(by="频率-4",ascending=False).reset_index()
df_tmp4 = good_bad_none_all[["频率-6","评论-1字"]].groupby("评论-1字").sum().sort_values(by="频率-6",ascending=False).reset_index()
df_tmp5 = good_bad_none_all[["频率-8","评论-2字"]].groupby("评论-2字").sum().sort_values(by="频率-8",ascending=False).reset_index()
df_tmp6 = good_bad_none_all[["频率-10","评论-3字"]].groupby("评论-3字").sum().sort_values(by="频率-10",ascending=False).reset_index()
df_all_temp = pd.concat([df_tmp1,df_tmp2,df_tmp3,df_tmp4,df_tmp5,df_tmp6],axis=1)
df_all_temp.to_excel(writer,sheet_name="汇总-"+str(len(good_bad_none_all)),index=None)
df_all = pd.DataFrame()
for file in files:
    keyname = file.split("-")[0]
    path = paths+file
    df = pd.read_excel(path)
    df = df[["标题","内容","星级"]]
    df_all = pd.concat([df_all,df])
df_all.sort_values(by="星级",ascending=False,inplace=True)
df_all.to_excel(writer,sheet_name="标题-评论-星级汇总-"+str(len(df_all)),index=None)
writer.close()

In [9]:
a = []
b = []
for i,j in enumerate(list(good_df_out.columns)):
    if i%2==0:
        a.append(j)
    else:
        b.append(j)
c = []
for i in range(int(len(list(good_df_out.columns))/2)):
    
    c.append(b[i])
    c.append(a[i])
c

['频率-0',
 '标题-1字',
 '频率-2',
 '标题-2字',
 '频率-4',
 '标题-3字',
 '频率-6',
 '评论-1字',
 '频率-8',
 '评论-2字',
 '频率-10',
 '评论-3字']

In [10]:
list(good_df_out.columns)

['标题-1字',
 '频率-0',
 '标题-2字',
 '频率-2',
 '标题-3字',
 '频率-4',
 '评论-1字',
 '频率-6',
 '评论-2字',
 '频率-8',
 '评论-3字',
 '频率-10']

In [11]:
def change_coloum(df):
    a,b,c = [],[],[]
    for i,j in enumerate(list(df.columns)):
        if i%2==0:
            a.append(j)
        else:
            b.append(j)
    for i in range(int(len(list(df.columns))/2)):
        c.append(b[i])
        c.append(a[i])
    df = df[c]
    return df

In [12]:
good_df_out

,标题-1字,频率-0,标题-2字,频率-2,标题-3字,频率-4,评论-1字,频率-6,评论-2字,频率-8,评论-3字,频率-10
0,great,357.0,plant stand,118.0,great plant stand,17.0,great,510.0,plant stand,216,easy put together,85.0
1,plant,197.0,five stars,114.0,easy put together,10.0,plant,490.0,easy assemble,156,super easy assemble,22.0
2,stand,187.0,great product,48.0,perfect plant stand,10.0,stand,390.0,put together,114,easy assemble looks,16.0
3,sturdy,172.0,plant stands,42.0,sturdy plant stand,7.0,sturdy,374.0,plant stands,91,love plant stand,13.0
4,nice,163.0,looks great,35.0,nice plant stand,7.0,easy,328.0,looks great,90,super easy put,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,timely manner,2,NaN,NaN
2311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,product provides,2,NaN,NaN
2312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,well wish,2,NaN,NaN
2313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pots flowers,2,NaN,NaN


In [13]:
change_coloum(good_df_out)

,频率-0,标题-1字,频率-2,标题-2字,频率-4,标题-3字,频率-6,评论-1字,频率-8,评论-2字,频率-10,评论-3字
0,357.0,great,118.0,plant stand,17.0,great plant stand,510.0,great,216,plant stand,85.0,easy put together
1,197.0,plant,114.0,five stars,10.0,easy put together,490.0,plant,156,easy assemble,22.0,super easy assemble
2,187.0,stand,48.0,great product,10.0,perfect plant stand,390.0,stand,114,put together,16.0,easy assemble looks
3,172.0,sturdy,42.0,plant stands,7.0,sturdy plant stand,374.0,sturdy,91,plant stands,13.0,love plant stand
4,163.0,nice,35.0,looks great,7.0,nice plant stand,328.0,easy,90,looks great,12.0,super easy put
...,...,...,...,...,...,...,...,...,...,...,...,...
2310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,timely manner,NaN,NaN
2311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,product provides,NaN,NaN
2312,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,well wish,NaN,NaN
2313,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,pots flowers,NaN,NaN
